In [ ]:
import os
import re
import bz2
import numpy as np
import pandas as pd
import seaborn as sns
import ROOT, array
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
def pulses(file_path):
  nl = 0
  bins = [[] for _ in range(33)]
  cont = 0
  clocks = 0
  try:
    with bz2.open(file_path, 'rt') as f: 
      for line in f:
        if line[0] != "#":
          if nl < 32:
            row_data = [int(num_str) for num_str in re.findall(r'-?\d+', line)]
            bins[nl].append(row_data[0])
          nl += 1
        if line[0] == "#":
          if line[2] == "t":
            row_data = [int(num_str) for num_str in re.findall(r'\d+', line)]
            bins[nl].append(row_data[1]-clocks)
            if row_data[1] < clocks:
              #print("Here")
              print(row_data[1],clocks)
            clocks = row_data[1]
            nl = 0
          if line[2:4] == "r2":
            clocks = 0
            #cont += 1
            #if cont == 2:
              #break
  except EOFError as e:
    print("Warning: Incomplete or corrupted file. Processing stopped early:", e)

  if nl != 0:
    for i in range(nl):
      if bins[i]:
        bins[i].pop()

  bins_np = np.array(bins)
  df = pd.DataFrame(bins_np.transpose(), columns=[int(i) for i in range(33)])
  df = df.rename(columns={32: "deltaTime"})
  return df

def Peak_Position(df,sat,bin_sat_position):
  Peak_Position = pd.to_numeric(df.iloc[:, 0:32].idxmax(axis=1))
  df.loc[:,'Peak_Position']= Peak_Position

  if sat == True:
    df.drop(df[df['Peak_Position'] >= bin_sat_position].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

def A3CAP(df):
    percentages = []
    for pulse in range(len(df.iloc[:,0])):
      peak = int(bins.loc[pulse,'Peak_Position'])
      mean = ( int(df.iloc[pulse,peak+1]) + int(df.iloc[pulse,peak+2])+ int(df.iloc[pulse,peak+3]))/3
      percentage = int((int(df.iloc[pulse,peak])-mean)*100/int(df.iloc[pulse,peak]))
      percentages.append(percentage)
    df.loc[:, 'A3CAP'] = percentages

def SPC_16(df):
    percentages = []
    for pulse in range(len(df.iloc[:,0])):
      peak = int(bins.loc[pulse,'Peak_Position'])
      percentage = int((int(df.iloc[pulse,peak])-int(df.iloc[pulse,peak+2]))*100/int(df.iloc[pulse,peak]))
      percentages.append(percentage)
    df.loc[:, 'SPC-16'] = percentages

def SPC_24(df):
    percentages = []
    for pulse in range(len(df.iloc[:,0])):
      peak = int(bins.loc[pulse,'Peak_Position'])
      percentage = int((int(df.iloc[pulse,peak])-int(df.iloc[pulse,peak+3]))*100/int(df.iloc[pulse,peak]))
      percentages.append(percentage)
    df.loc[:, 'SPC-24'] = percentages
    
def SPC_32(df):
    percentages = []
    for pulse in range(len(df.iloc[:,0])):
      peak = int(bins.loc[pulse,'Peak_Position'])
      percentage = int((int(df.iloc[pulse,peak])-int(df.iloc[pulse,peak+4]))*100/int(df.iloc[pulse,peak]))
      percentages.append(percentage)
    df.loc[:, 'SPC-32'] = percentages


def Charge_Histogram(df):
  charge = np.array(df['charge'])
  counts, bin_edges= np.histogram(charge, bins=np.arange(min(charge),max(charge) + 1, 7))
  bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
  plt.figure(figsize=(15, 8))
  #plt.semilogy(bin_centers, counts, '-o', color='blue')
  plt.semilogy(bin_centers, counts, color='blue')
  plt.ylabel("# of Counts", fontsize=20)
  plt.xlabel("ADC Values", fontsize=20)
  plt.title("Integraded_Charge_Histogram", fontsize=20)
  plt.legend(fontsize=20)
  plt.grid(True)


def charge(df):
    charge = []
    charge = df.iloc[:,0:32].sum(axis=1)
    df.loc[:, 'Charge'] = charge


In [ ]:
# Extractiong information every three hours
dfs = []

base_date = '2024_06_01'
base_path = '/eos/user/d/dmerizal/SWAN_projects/LAGO/Datos_ICRC/chiapas_data/'

for hour in range(21,24):  # desde 00 hasta 06 inclusive
    hour_str = f"{hour:02d}"  # formato '00', '01', ..., '06'
    file_path = f"{base_path}jaguarito_nogps_{base_date}_{hour_str}h00.dat.bz2"
    print(file_path)
    df_hour = pulses(file_path)
    dfs.append(df_hour)

df_total = pd.concat(dfs, ignore_index=True)

df_total.to_csv("jaguarito_nogps_2024_06_01_21h00-23h00.csv")

In [ ]:
# df for twenty-four hour
dfs = []
#jaguarito_nogps_2025_01_01_00h00.dat.bz2
#jaguarito_nogps_2023_10_25_00h00.dat.bz2
#jaguarito_nogps_2024_06_01_00h00.dat.bz2

base_date = '2024_06_01'
hours_1 = [0,4,8,12,16,21]
hours_2 = [3,7,11,15,20,23]
for i in range(0,6):  # desde 00 hasta 06 inclusive
    hour_str_1 = f"{hours_1[i]:02d}"  # formato '00', '01', ..., '06'
    hour_str_2 = f"{hours_2[i]:02d}"
    file_path = f"jaguarito_nogps_{base_date}_{hour_str_1}h00-{hour_str_2}h00_Pulses.csv"
    print(file_path)
    df_hour = pd.read_csv(file_path)
    dfs.append(df_hour)

df_total = pd.concat(dfs, ignore_index=True)

df_total.to_csv("jaguarito_nogps_2024_06_01_dia.csv")
bins.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
bins.drop('Unnamed: 0', axis=1, inplace=True)
Peak_Position(bins,True,20)
A3CAP(bins)
SPC_16(bins)
SPC_24(bins)
SPC_32(bins)
charge(bins)
bins.to_csv("jaguarito_nogps_2024_06_01_04h00-07h00_Pulses.csv")

In [ ]:
data = pd.read_csv("Raw_data.csv")

In [ ]:
data.keys()

In [ ]:
def make_fit(delta_times):
   
    counts, edges = np.histogram(
        delta_times,
        bins=np.arange(delta_times.min(), delta_times.max() + 100, 100)
    )
    centers = (edges[:-1] + edges[1:]) / 2

    h = ROOT.TH1D("h", "Histogram", len(counts), edges[0], edges[-1])
    for i, val in enumerate(counts):
        h.SetBinContent(i + 1, val)
        h.SetBinError(i + 1, max(1.0, np.sqrt(val)))


    fit_func = ROOT.TF1("fit_func", "[0]*exp(-x/[1]) + [2]", edges[0], edges[-1])
    fit_func.SetParNames("A", "Tau", "B")
    fit_func.SetParameters(max(counts), 4000, min(counts))

    fit_result = h.Fit(fit_func, "RQ")

    tau = fit_func.GetParameter(1)
    chi2 = fit_func.GetChisquare()
    ndf = fit_func.GetNDF()
    chi2_red = chi2 / ndf if ndf != 0 else float('inf')

    return tau, chi2_red

def multiple_fit(df, percentage):
    results = []

    filters = {
        "A3CAP": df['A3CAP'] < percentage,
        "SPC-16": df['SPC-16'] < percentage,
        "SPC-24": df['SPC-24'] < percentage,
        "SPC-32": df['SPC-32'] < percentage,
    }

    for label, mask in filters.items():
        delta_times = df.loc[mask, 'deltaTime'].to_numpy()

        if len(delta_times) == 0:
            results.append({'Method': label, 'percentage': percentage, 'Tau': np.nan, 'Chi2_Red': np.nan})
            continue

        tau, chi2_red = make_fit(delta_times)
        results.append({'Method': label, 'percentage': percentage, 'Tau': round(tau, 1), 'Chi2_Red': round(chi2_red, 2)})

    return results

# Evaluar para varios percentages
percentages = list(range(10, 101, 10))

all_results = []

for p in percentages:
    results_p = multiple_fit(data,p)  # bins es tu DataFrame original
    all_results.extend(results_p)

# Crear DataFrame final
df_all = pd.DataFrame(all_results)

# Opcional: mostrar
print(df_all.tail(20))

In [ ]:
sns.set(style="whitegrid", context="talk", palette="colorblind")

plt.figure(figsize=(20, 10))

# Usar scatterplot para mostrar solo puntos
sns.scatterplot(
    data=df_all,
    x="percentage",
    y="Tau",
    hue="Method",
    style="Method",
    s=400,  # tamaño del punto
)
plt.axhline(y=2200, color='black', linestyle='--', linewidth=2, label=r'$\tau_\mu = 2200$ ns')

plt.xlabel("Distance to Peak Cutoff[%]", fontsize=50)
plt.ylabel(r" Fit $\tau$ Value  [ns]", fontsize=50)
plt.xticks(range(10,101,10),fontsize=30)
plt.yticks(fontsize=30)
plt.legend(title="Algorithms",fontsize=30,markerscale=1,title_fontsize=30, loc='upper left', bbox_to_anchor=(1.01, 1))
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()


plt.savefig("tau_vs_percentage_points.png", dpi=300)
plt.show()


In [ ]:
mask = (data['SPC-24'] < 60) &  (data['deltaTime'] < 25000) &  (data['deltaTime'] > 500)
delta_times = data.loc[mask, 'deltaTime'].to_numpy()

counts, edges = np.histogram(delta_times,
                             bins=np.arange(delta_times.min(), delta_times.max() + 1, 100))
centers = (edges[:-1] + edges[1:]) / 2


t_root = array.array('d', centers.astype('float64'))
N_root = array.array('d', counts.astype('float64'))
errors = array.array('d', [max(1.0, np.sqrt(n)) for n in counts])


gre = ROOT.TGraphErrors(len(t_root), t_root, N_root,
                        array.array('d', [0]*len(t_root)), errors)
gre.SetTitle("SPC-24;Time Difference [ns];# of Entries/100")
gre.SetMarkerStyle(20)


c1 = ROOT.TCanvas("c1", "Ajuste", 800, 600)
gre.Draw("AP")


fit_func = ROOT.TF1("fit_func", "[0]*exp(-x/[1]) + [2]", 100, 25000)
fit_func.SetParNames("A", "#tau_{{#mu}}", "B")
fit_func.SetParameters(max(counts), 40000, min(counts))  # valores iniciales


gre.Fit(fit_func, "R")


A = fit_func.GetParameter(0)
tau_mu = fit_func.GetParameter(1)
B = fit_func.GetParameter(2)
err_A = fit_func.GetParError(0)
err_mu = fit_func.GetParError(1)
err_B = fit_func.GetParError(2)

chi2 = fit_func.GetChisquare()
ndf = fit_func.GetNDF()
chi2_red = chi2 / ndf


legend = ROOT.TPaveText(0.65, 0.7, 0.88, 0.88, "NDC")  
legend.SetFillColor(0)
legend.SetBorderSize(1)
legend.SetTextFont(42)
legend.AddText(f"#chi^{{2}} / ndf   {chi2:.0f} / {ndf}")
legend.AddText(f"A        {A:.1f} #pm {err_A:.1f}")
legend.AddText(f"#tau_{{#mu}}  {tau_mu:.1f} #pm {err_mu:.1f} ns")

legend.AddText(f"B        {B:.1f} #pm {err_B:.1f}")
legend.Draw()

# --- Guardar ---
c1.SaveAs("Exponential_Fit.png")
c1.Update()
input("Press Enter to close...")